<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/characters/novel_collecting/ChatBot%E6%8A%BD%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implemented by 李鲁鲁

- [x] 读取总结之后输出的jsonl
- [x] 总结所有角色出现的频率，可视化
- [x] 想个办法截取特定角色
- [x] 如果太长则切一下

TODO 米唯实：

- [ ] 把role_name和几个参数的设计移动到脚本前面来
- [ ] 支持target_names设置多个人物
- [ ] 如果不设置，自动整理出现最多的非scene的3个人物
- [ ] 把抽取后的三个文本，整理成txt和jsonl的那个notebook上传到github的文件夹
- [ ] 把整理后的这个脚本 ，也上传到这个文件夹
- [ ] 在readme，详细说明小说抽取、整合、和chatbot提取的用法

https://github.com/LC1332/Chat-Haruhi-Suzumiya/tree/main/characters/novel_collecting

这里



input_name = '/content/reorganized_xiaoaojianghu.jsonl'

用utf8编码为我读取jsonl文件，并且把每一行解析成一个list of json, data

请用python为我实现

In [5]:
input_name = '/content/reorganized_xiaoaojianghu.jsonl'

import json

data_in_chunk = []
with open(input_name, encoding='utf8') as f:
    for line in f:
        data_in_chunk.append(json.loads(line))

data = []

for chunk in data_in_chunk:
    for d in chunk:
        data.append(d)

print(len(data))

16221


data是一个list of json，里面有一个字段是role

为我统计role里面各个字段出现的频率，并输出出现最多的30个角色和对应的频率

In [13]:
from collections import Counter

role_counts = Counter()
for line in data:
    role = line['role']
    role_counts[role] += 1

common_roles = role_counts.most_common(30)

status = 0

role_name = []

for role, count in common_roles:
    status = status + 1
    if role != 'scene':
        role_name.append(role)
    if status % 3 == 0:
        print(role, count)
    else:
        print(role, count, end=' ')


scene 6172 令狐冲 2558 岳不群 434
岳灵珊 422 盈盈 378 林平之 356
仪琳 329 田伯光 217 任我行 198
向问天 185 岳夫人 168 史镖头 149
林震南 128 婆婆 121 左冷禅 109
方证 105 陆大有 97 余沧海 97
桃根仙 93 桃花仙 83 仪和 82
劳德诺 81 刘正风 81 桃枝仙 81
定静师太 81 祖千秋 77 黑白子 75
桃干仙 74 曲非烟 66 丹青生 66


In [14]:
from IPython.core.interactiveshell import default_banner
role_in_time = []

dialogue_in_time = []



for d in data:
    role = d['role']
    if role == 'scene':
        continue

    if role in role_name:

        role_in_time.append(role)
        dialogue_in_time.append(d['text'])



In [16]:
import plotly.express as px

# 创建映射字典,角色名字到编号
name_to_num = {name:i for i,name in enumerate(role_name)}

# 将角色名字序列转换为编号序列
role_index = [name_to_num[name] for name in role_in_time]

# 生成不同颜色的颜色序列
colors = px.colors.qualitative.Dark24
while len(colors) < len(role_name):
    colors = colors + colors


# 绘制散点图
fig = px.scatter(
    x=list(range(len(role_in_time))),
    y=role_index,
    color=[colors[i] for i in role_index],
    hover_data={'t': dialogue_in_time}
)

# 设置y轴为角色名字
fig.update_yaxes(ticktext=role_name, tickvals=list(range(len(role_name))))


# 设置坐标轴范围
fig.update_xaxes(range=[0, len(role_in_time)])
# fig.update_yaxes(range=[0, len(role_name)])

# 设置坐标轴标题
fig.update_layout(
    xaxis_title='时间',
    yaxis_title='角色'
)

fig.show()

新的逻辑非常简单，我们设置一个max_find_lines = 10

支持跨越多少行寻找目标角色。

如果找不到，就给截断了。然后对于连续的，我做一个切分。

In [49]:
max_find_lines = 10

max_token_num = 500

target_role = '令狐冲'

first_scene_id = 0

for i,d in enumerate(data):
    if d['role'] == 'scene':
        first_scene_id = i
        break

chat_ids = []

# 先寻找所有出现令狐冲的节点
for i,d in enumerate(data):
    if d['role'] == target_role:
        chat_ids.append(i)

previous_scene_ids = []

# 对于每一个chat_ids，向前寻找scene的节点
for chat_id in chat_ids:
    ans = first_scene_id
    for j in range(chat_id, first_scene_id,-1):
        if data[j]['role'] == 'scene':
            ans = j
            break
    previous_scene_ids.append(ans)



chat_ids是一个list of int。代表chat出现的顺序，是个严格升序的序列

我希望找到chat_ids中，所有的元素之间间隔不大于max_find_lines = 10连续子串，用list of list of int的形式，保存到chat_ids_in_chunk

例子输入
1,2,4,100,110,120,555
例子输出
[1,2,4],[100,110,120],[555]

请用python为我实现

In [28]:
chat_ids_in_chunk = []
current_chunk = []

for chat_id in chat_ids:
    if not current_chunk:
        current_chunk.append(chat_id)
        continue

    if chat_id - current_chunk[-1] <= max_find_lines:
        current_chunk.append(chat_id)
    else:
        chat_ids_in_chunk.append(current_chunk)
        current_chunk = [chat_id]

if current_chunk:
    chat_ids_in_chunk.append(current_chunk)

# print(chat_ids_in_chunk)

In [29]:
for chunk in chat_ids_in_chunk:
    print(len(chunk),end = ' ')


5 4 57 12 1 1 5 2 20 12 6 17 9 84 35 47 1 1 140 75 4 1 5 2 5 2 11 1 5 46 7 3 5 4 1 15 20 5 2 1 1 7 3 2 1 30 10 1 2 37 4 1 1 7 1 16 3 1 2 7 19 3 11 1 1 1 1 1 2 1 2 12 4 2 3 1 5 30 67 65 6 59 4 8 9 1 14 2 15 4 1 1 3 14 6 1 11 2 10 2 45 15 26 7 15 3 2 6 15 15 1 1 17 19 8 9 23 5 1 1 23 15 1 1 1 3 55 14 1 2 3 3 7 2 9 37 3 2 4 30 4 13 2 31 1 2 19 1 2 1 2 2 1 7 3 50 3 1 8 14 21 5 25 12 4 8 67 1 2 19 1 1 2 1 7 1 1 5 19 7 7 3 1 1 2 1 4 1 1 3 1 3 3 1 1 2 6 9 1 4 2 1 3 2 1 1 5 2 6 6 6 1 24 6 2 27 1 1 10 2 20 1 2 2 20 6 1 2 7 3 5 3 27 1 1 3 5 5 1 31 19 5 51 22 18 1 1 5 6 4 38 4 4 7 16 

In [66]:
print(chat_ids_in_chunk[0])

chat_id2previous_scene_id = {}

for previous, chat_id in zip(previous_scene_ids, chat_ids):
    chat_id2previous_scene_id[chat_id] = previous
    if previous > 0:
        if data[previous-1]['role'] != target_role:
            chat_id2previous_scene_id[chat_id] -= 1



[1715, 1723, 1727, 1731, 1732]


这里我们先计算一下每一句所花的token数量

In [37]:
def data2str( data ):
    role = data['role']
    if role in ['旁白', '', 'scene','Scene','narrator' , 'Narrator']:
        return 'scene:' + data['text']
    else:
        return role + ':「' + data['text'] + '」'

In [38]:
!pip install -q tiktoken

import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

def count_token( my_str ):
    return len(enc.encode(my_str))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.6 MB/s eta 0:00:00


In [39]:
line_token = [count_token(data2str(d)) for d in data]
print(line_token[:5])

[17, 26, 21, 46, 39]


In [40]:
print(sum(line_token[:5]))

149


我们现在需要把这东西变成最终的texts文本

In [67]:
from ast import Break
final_chunks = []

print_count = 0



for chunk in chat_ids_in_chunk:
    N = len(chunk)

    current_i = 0

    while current_i < N-1:



        consider_chat_id = chunk[current_i]

        previous_scene_id = chat_id2previous_scene_id[consider_chat_id]

        #保底
        withdraw_start = previous_scene_id
        withdraw_end = consider_chat_id

        current_count = sum(line_token[previous_scene_id:consider_chat_id+1])
        while current_count < max_token_num and current_i < N-1:
            consider_end = chunk[current_i + 1]
            consider_count = sum(line_token[previous_scene_id:consider_end+1])
            if consider_count < max_token_num:
                current_count = consider_count
                withdraw_start = previous_scene_id
                withdraw_end = consider_end
                current_i += 1
            else:
                break

        # print_count += 1

        # print(withdraw_start, end = ' ')
        # if print_count % 5 == 0:
        #     print()

        chunk_str = ''
        for i in range(withdraw_start, withdraw_end+1):
            chunk_str += data2str(data[i]) + '\n'

        current_i += 1

    final_chunks.append(chunk_str)


In [68]:
print(len(final_chunks))

255


In [69]:
import os
import zipfile

os.makedirs('/content/texts', exist_ok=True)

for i in range(0,len(final_chunks)):
  my_str = final_chunks[i]
  with open(f'/content/texts/text_{i}.txt', 'w', encoding='utf-8') as f:
    f.write(my_str)

zip_path = '/content/linghuchong_text.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
  for filename in os.listdir('/content/texts'):
    zipf.write('/content/texts/' + filename)

print('Zipped folder saved to', zip_path)

Zipped folder saved to /content/linghuchong_text.zip
